In [1]:
from model import build_model
from config import Dataset, ClassicalModel, Embedding, Pooling, gen_args
from train import get_flops
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from sklearn.model_selection import ShuffleSplit
from fvcore.nn import FlopCountAnalysis
import torch

/home/jsimonrichard/QML_Research/QFE-Experiments/qfe_exp_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args = gen_args(Dataset.PROTEINS, Embedding.QFE_EXP, ClassicalModel.GCN, Pooling.MEAN)

In [3]:
ds = TUDataset(root="data", name=args.dataset.value, use_node_attr=True)
rs = ShuffleSplit(n_splits=1, test_size=0.1, random_state=args.seed)
train_index, test_index = next(rs.split(ds))
train_ds = ds[train_index]

In [4]:
len(test_index)

112

In [15]:
model = build_model(args, 4, 2)

In [16]:
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True)

In [17]:
case = next(iter(train_loader))
x = case.x
edge_index = case.edge_index
batch = case.batch

In [18]:
batch.shape

torch.Size([208])

In [19]:
x.size(0)

208

In [20]:
get_flops(model, "cpu", x, edge_index, torch.zeros(batch.size(0), dtype=torch.int64))

IndexError: list index out of range

In [17]:
FlopCountAnalysis(model, inputs=(
    x,
    edge_index,
    batch
)).total()

Unsupported operator aten::ne encountered 8 time(s)
Unsupported operator aten::repeat encountered 8 time(s)
Unsupported operator aten::alias encountered 8 time(s)
Unsupported operator aten::expand_as encountered 18 time(s)
Unsupported operator aten::scatter_add_ encountered 18 time(s)
Unsupported operator aten::pow_ encountered 8 time(s)
Unsupported operator aten::masked_fill_ encountered 8 time(s)
Unsupported operator aten::mul encountered 24 time(s)
Unsupported operator aten::index_select encountered 8 time(s)
Unsupported operator aten::add encountered 8 time(s)
Unsupported operator aten::new_ones encountered 1 time(s)
Unsupported operator aten::div encountered 1 time(s)
Unsupported operator aten::log_softmax encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their 

698880